In [1]:
from modules.base import *

#### 1. 요약 사용하기

In [22]:
class State(MessagesState):
    summary: str

@trace_function(enable_print=True)
def conversation(state: State, 
                 config: RunnableConfig):
    
    summary = state.get("summary", "")
    if summary:
        system_message = f"다음은 지금까지의 대화 요약입니다. {summary}\n\n"
        messages = [SystemMessage(content=system_message)] + state["messages"]
    else:
        messages = state["messages"]
    response = llm.invoke(messages, config)
    return {"messages": response}

@trace_function(enable_print=True)
def summarize_conversation(state: State):
    summary = state.get("summary", "")
    if summary:
        summary_message = (
            f"다음은 지금까지의 대화 요약입니다. {summary}\n\n"
            "새로운 메시지를 고려하여 요약 내용을 업데이트하세요."
        )
    else:
        summary_message = "위 내용을 요약하세요."
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = llm.invoke(messages)
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}

@trace_function(enable_print=True)
def should_continue(state: State):    
    messages = state["messages"]
    if len(messages) >= 6:
        print(f"{YELLOW}\n대화 길이가 6 이상 (3번 이상 대화 주고받음) 이므로 요약 노드로 이동합니다.\n{RESET}")
        return "summarize_conversation"
    return END

workflow = StateGraph(State)
workflow.add_node("conversation", conversation)
workflow.add_node("summarize_conversation", summarize_conversation)
workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_conversation", END)
memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)

In [23]:
config = {"configurable": {"thread_id": "first_chat", 
                           "user_id": "changwoo"}}

In [24]:
_ = graph.invoke({"messages":"안녕 나는 창우라고해"}, config=config)


🚀 Passing Through [conversation] ..

#### [Input State]
  args: ({'messages': [HumanMessage(content='안녕 나는 창우라고해', additional_kwargs={}, response_metadata={}, id='c4d2cabc-e1cf-4b2d-bb90-ea0ad363a071')]},)
  kwargs: {'config': {'metadata': {'thread_id': 'first_chat', 'user_id': 'changwoo', 'langgraph_step': 1, 'langgraph_node': 'conversation', 'langgraph_triggers': ['start:conversation'], 'langgraph_path': ('__pregel_pull', 'conversation'), 'langgraph_checkpoint_ns': 'conversation:f8b0bf6b-85c2-4775-7ffd-50d8fcec9e8e'}, 'configurable': {'thread_id': 'first_chat', 'user_id': 'changwoo', '__pregel_resuming': False, '__pregel_task_id': 'f8b0bf6b-85c2-4775-7ffd-50d8fcec9e8e', '__pregel_send': functools.partial(<function PregelRunner.tick.<locals>.writer at 0x7fc6af8cad40>, PregelExecutableTask(name='conversation', input={'messages': [HumanMessage(content='안녕 나는 창우라고해', additional_kwargs={}, response_metadata={}, id='c4d2cabc-e1cf-4b2d-bb90-ea0ad363a071')]}, proc=<langgraph.utils.runnable.

In [25]:
_ = graph.invoke({"messages":"나는 올해 30살이고, 현재 부산대학교에서 박사과정을 하고있어."}, config=config)


🚀 Passing Through [conversation] ..

#### [Input State]
  args: ({'messages': [HumanMessage(content='안녕 나는 창우라고해', additional_kwargs={}, response_metadata={}, id='c4d2cabc-e1cf-4b2d-bb90-ea0ad363a071'), AIMessage(content='안녕하세요, 창우님! 만나서 반갑습니다. 오늘 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 14, 'total_tokens': 36, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_5f20662549', 'finish_reason': 'stop', 'logprobs': None}, id='run-41d3b35e-928e-41ff-bc86-b090e551e850-0', usage_metadata={'input_tokens': 14, 'output_tokens': 22, 'total_tokens': 36, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessage(content='나는 올해 30살이고, 현재 부

In [26]:
_ = graph.invoke({"messages":"내 전공은 인공지능이고, 요즘 LLM 분야에 관심이 많아."}, config=config)


🚀 Passing Through [conversation] ..

#### [Input State]
  args: ({'messages': [HumanMessage(content='안녕 나는 창우라고해', additional_kwargs={}, response_metadata={}, id='c4d2cabc-e1cf-4b2d-bb90-ea0ad363a071'), AIMessage(content='안녕하세요, 창우님! 만나서 반갑습니다. 오늘 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 14, 'total_tokens': 36, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_5f20662549', 'finish_reason': 'stop', 'logprobs': None}, id='run-41d3b35e-928e-41ff-bc86-b090e551e850-0', usage_metadata={'input_tokens': 14, 'output_tokens': 22, 'total_tokens': 36, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessage(content='나는 올해 30살이고, 현재 부